In [1]:
!pip install tensorflow==2.4.0

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2, MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model

In [2]:
batch_size = 32
img_height = 244
img_width = 244
input_shape = (img_width,img_height)
num_classes=2

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data/train',
  validation_split=None,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data/val',
  validation_split=None,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))



Found 86744 files belonging to 2 classes.
Found 10954 files belonging to 2 classes.


In [3]:
def my_net(model,freeze_layers=10,full_freeze='N'):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(2,activation='sigmoid')(x)
    model_final = Model(model.input,out)
    if full_freeze != 'N':
        for layer in model.layers[0:freeze_layers]:
            layer.trainable = False
    return model_final

In [4]:
#model = ResNet50V2(weights='imagenet',include_top=False)
#model = my_net(model)

In [5]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(244,244,3), alpha=1.0, include_top=False,
    input_tensor=None, pooling=None, classes=2,
    classifier_activation='softmax'
)
model = my_net(model)

In [6]:
#model = tf.keras.applications.VGG16(
#    include_top=True, weights=None, input_tensor=None,
#    input_shape=(244, 244, 3), pooling=None, classes=2,
#    classifier_activation='softmax'
#)
#model = my_net(model)

In [7]:
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=2,
    validation_data=val_ds
)

Epoch 1/2
2711/2711 [==============================] - 8434s 3s/step - loss: 0.4579 - accuracy: 0.7795 - val_loss: 9.9463 - val_accuracy: 0.5378
Epoch 2/2
2711/2711 [==============================] - 8198s 3s/step - loss: 0.2872 - accuracy: 0.8762 - val_loss: 1.0902 - val_accuracy: 0.8041


In [9]:
!rm -Rf model

In [10]:
model.save('model')


INFO:tensorflow:Assets written to: model/assets
